In [7]:
import numpy as np
import json
import cv2
import os

In [3]:
# points_folder = "./extracted_points/thermal"
points_folder = "./extracted_points/visible"

# image_folder = "./calibration_data/thermal"
image_folder = "./calibration_data/visible"

# Load calibration params
# params_json = "./thermal_params.json"
params_json = "./visible_params.json"
with open(params_json, 'r') as f:
	params = json.load(f)
	f.close()

# Save dewarped points
undistorted_points = dict()

points_files = os.listdir(points_folder)
for pnt_file in points_files:
	# Load image
	img_file = (os.listdir(image_folder)[0]).split(".")[-1]
	img_file = pnt_file.replace(".csv", "." + img_file)
	img = cv2.imread(os.path.join(image_folder, img_file), cv2.IMREAD_UNCHANGED)

	# Dewarp image
	# img = cv2.undistort(img, np.array(params["mtx"]), np.array(params["dist"]), None, np.array(params["newcameramtx"]))
	img = cv2.undistort(img, np.array(params["mtx"]), np.array(params["dist"]))

	# Load points
	src = np.genfromtxt(os.path.join(points_folder, pnt_file), delimiter=",")

	# Dewarp points
	map1, map2 = cv2.initInverseRectificationMap(
		np.array(params["mtx"]), 
		np.array(params["dist"]), 
		None, 
		# np.array(params["newcameramtx"]), 
		np.array(params["mtx"]), 
		(img.shape[1], img.shape[0]), 
		cv2.CV_32FC1
	)

	# Approximately interpolate src points
	scale = 1
	y, x = [int(round(scale*v)) for v in map1.shape[:2]]
	big_map1 = cv2.resize(map1, (x, y), interpolation=cv2.INTER_CUBIC)
	big_map2 = cv2.resize(map2, (x, y), interpolation=cv2.INTER_CUBIC)

	dst_x = [big_map1[*np.round((scale*p[1], scale*p[0])).astype("int32")] for p in src]
	dst_y = [big_map2[*np.round((scale*p[1], scale*p[0])).astype("int32")] for p in src]
	dst = list(zip(dst_x, dst_y))
	undistorted_points[pnt_file] = dst

	# Draw points
	for p in dst:
		x, y = np.round(p).astype("int32")
		img = cv2.circle(img, (x,y), 5, (0, 0, 255), -1)

	# Show points
	cv2.namedWindow("img", cv2.WINDOW_NORMAL)
	cv2.imshow("img", img)
	while cv2.waitKey(0) != ord('q'): continue
	cv2.destroyAllWindows()


In [6]:
# Save dewarped image points
dst_folder = os.path.join("undistorted_points/", os.path.split(points_folder)[-1])
if not os.path.isdir(dst_folder): os.mkdir(dst_folder)

for fname in undistorted_points:
	arr = np.array(undistorted_points[fname])
	arr = np.flip(arr, 0) # Reverse visible array
	np.savetxt(os.path.join(dst_folder, fname), arr, fmt="%.10f", delimiter=",")

In [33]:
# Compute Homography
H = None
fnames = os.listdir("./undistorted_points/thermal/")
for f in fnames:
	# Load source and destination points
	src_points = np.genfromtxt(os.path.join("./undistorted_points/thermal/", f), delimiter=",")
	dst_points = np.genfromtxt(os.path.join("./undistorted_points/visible/", f), delimiter=",")

	# Find homography
	new_h = cv2.findHomography(src_points, dst_points)[0]
	
	if type(H) == type(None): H = new_h
	else: H = np.dstack((H, new_h))

# Compute mean
H = np.mean(H, axis=2)
H

array([[-5.00522230e+00,  2.20080159e-01,  7.43539331e+02],
       [-7.78015405e-02, -4.98586515e+00,  5.57117487e+02],
       [ 1.52584180e-04,  3.69336253e-04,  1.00000000e+00]])

In [36]:
# Load paramaters
with open("./thermal_params.json", 'r') as f:
	params = json.load(f)
	thermal_mtx  = np.array(params["mtx"])
	thermal_dist = np.array(params["dist"])
	f.close()

with open("./visible_params.json", 'r') as f:
	params = json.load(f)
	visible_mtx  = np.array(params["mtx"])
	visible_dist = np.array(params["dist"])
	f.close()

thermal_dir = "./calibration_data/thermal"
visible_dir = "./calibration_data/visible"

thermal_names = os.listdir(thermal_dir)
visible_names = os.listdir(visible_dir)
for i in range(len(thermal_names)):
	assert thermal_names[i].split(".")[0] == visible_names[i].split(".")[0]

	# Load Warped Images
	thermal_img = cv2.imread(os.path.join(thermal_dir, thermal_names[i]), cv2.IMREAD_UNCHANGED)
	visible_img = cv2.imread(os.path.join(visible_dir, visible_names[i]), cv2.IMREAD_UNCHANGED)
	
	# Undistort images
	thermal_img = cv2.undistort(thermal_img, thermal_mtx, thermal_dist)
	visible_img = cv2.undistort(visible_img, visible_mtx, visible_dist)

	# Warp thermal image
	h, w = visible_img.shape[:2]
	thermal_warped = cv2.warpPerspective(thermal_img, H, (w, h))

	# Overlay Images
	combined = cv2.addWeighted(visible_img, 1, thermal_warped, 0.5, 0)

	# Show images
	cv2.namedWindow("visible", cv2.WINDOW_NORMAL)
	cv2.namedWindow("thermal", cv2.WINDOW_NORMAL)

	cv2.imshow("visible", visible_img)
	cv2.imshow("thermal", thermal_warped)

	while cv2.waitKey(0) != ord('q'): continue
	cv2.destroyAllWindows()
